# Zipline 交易限制

在def initialize(context): 裡面，可以加入六種限制：

    set_do_not_order_list(restricted_list, on_error='fail')
    set_long_only(on_error='fail')
    set_max_leverage(max_leverage)
    set_max_order_count(max_count, on_error='fail')
    set_max_order_size(asset=None, max_shares=None, max_notional=None, on_error='fail')
    set_max_position_size(self, asset=None, max_shares=None, max_notional=None, on_error='fail')
    
可以一次加入多個機制，on_error有兩種選項，'fail' 和 'log'，前者直接中斷程式並顯示錯誤訊息，後者會照樣執行但記錄錯誤。

參考資料：https://zipline.ml4trading.io/api-reference.html#trading-controls

## 設定環境

In [11]:
import pandas as pd
import datetime
import tejapi
import time
import os
import warnings
warnings.filterwarnings('ignore')

# tej_key-------------------------------------------
tej_key ='your key'
tejapi.ApiConfig.api_key = tej_key  
os.environ['TEJAPI_BASE'] = "https://api.tej.com.tw"
os.environ['TEJAPI_KEY'] = tej_key

# universe and benchmark----------------------------
idx=['TWN50']   # 'TM100' 'TWN50'
benchmark=['Y9997']

# date----------------------------------------------
# set date
start='2018-07-24'
end='2018-08-24'
os.environ['mdate'] = '20180724 20180824'      # start+' '+end #'20221011 20221223'

tz = 'UTC'
start_dt, end_dt = pd.Timestamp(start, tz = tz), pd.Timestamp(end, tz = tz)
# calendar------------------------------------------
calendar_name='TEJ'  # US equities  XTAI

# bundle_name---------------------------------------
bundle_name = 'tquant'

from zipline.utils.calendar_utils import get_calendar
if get_calendar(calendar_name).is_session(start_dt)==False:
    start_dt=get_calendar(calendar_name).next_open(start_dt)
    
if get_calendar(calendar_name).is_session(end_dt)==False:
    end_dt=get_calendar(calendar_name).previous_close(pd.Timestamp(end_dt))
    
from zipline.api import    *

from zipline import run_algorithm  
from zipline.finance import commission, slippage
from zipline.pipeline import Pipeline, CustomFactor
from zipline.pipeline.factors import Returns, AverageDollarVolume

from zipline.utils.run_algo import (get_transaction_detail,
                    get_data_for_alphalens,
                    get_record_vars)

from zipline.sources.TEJ_Api_Data import (get_Treasury_Return,
                                          get_Benchmark_Return)


from logbook import Logger, StderrHandler, INFO
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

from zipline.utils import run_algo
from zipline.data import bundles

log_handler = StderrHandler(format_string='[{record.time:%Y-%m-%d %H:%M:%S.%f}]: ' +
                            '{record.level_name}: {record.func_name}: {record.message}',
                            level=INFO)
log_handler.push_application()
log = Logger('Algorithm')

coid = "2330 1216 1101 IR0001 2317 5844 2454 2357"
    
os.environ['ticker'] = coid       #'1101 1102'   #coid

!zipline ingest -b tquant

Merging daily equity files:


[2023-08-15 01:09:39.497634] INFO: zipline.data.bundles.core: Ingesting tquant.


# set_do_not_order_list

### zipline.api.set_do_not_order_list(self, restricted_list, on_error='fail')
- restricted_list (container[Asset], SecurityList) – The assets that cannot be ordered.

## 1

我們把1101加入限制清單，on_error (發生時處理方法) 用'log'，然後設定在第三天下單1101，可以看到執行時跳出
    
    ERROR: handle_violation: Order for 100 shares of Equity(0 [1101]) at 2018-07-26 05:30:00+00:00 violates trading constraint 
    RestrictedListOrder({})

但是看到圖三transactions，照樣買入了1101。



In [13]:
def initialize(context):
    context.i = 0
    set_do_not_order_list(restricted_list = [symbol('1101')], on_error='log')
    set_slippage(slippage.FixedSlippage(spread = 0.0))
    set_commission(commission.PerDollar(cost=commission_cost))
    set_benchmark(symbol('IR0001'))
    
def handle_data(context, data):
    
    if context.i == 0:
        order(symbol('2330'), 100)
        
    if context.i == 2:
        order(symbol('1101'), 100)

    if context.i == 4:
        order(symbol('1216'), 100)
        
    context.i += 1
    
def analyze(context, perf):

    pass

commission_cost = 0.001425
capital_base = 1e6
treasury_returns = get_Treasury_Return(start = start_dt,
                                      end = end_dt,
                                      rate_type = 'Time_Deposit_Rate',                     
                                      term = '1y',
                                      symbol = '5844')

performance = run_algorithm(start=start_dt,
                            end=end_dt,
                            initialize=initialize,
                            handle_data=handle_data,
                            capital_base=capital_base,
                            analyze=analyze,
                            treasury_returns=treasury_returns,
                            trading_calendar=get_calendar(calendar_name),
                            bundle=bundle_name)

positions, transactions, orders = get_transaction_detail(performance)

[2023-08-15 01:09:48.064215]: ERROR: handle_violation: Order for 100 shares of Equity(0 [1101]) at 2018-07-26 05:30:00+00:00 violates trading constraint RestrictedListOrder({})
[2023-08-15 01:09:48.102672]: INFO: earn_dividends: Equity(1 [1216]), cash_dividend amount: 5.5, pay_date: 2018-09-07, div_owed: 550.0
[2023-08-15 01:09:48.193456]: INFO: handle_simulation_end: Simulated 24 trading days
first open: 2018-07-24 01:01:00+00:00
last close: 2018-08-24 05:30:00+00:00


In [14]:
transactions

,,,amount,dt,price,order_id,commission
date,asset,symbol,,,,,
2018-07-25 00:00:00+08:00,Equity(3 [2330]),2330,100,2018-07-25 13:30:00+08:00,240.5,d94228f5ca6246ebad9c18acdc642a50,None
2018-07-27 00:00:00+08:00,Equity(0 [1101]),1101,100,2018-07-27 13:30:00+08:00,40.3,87de752425fd4f9493fecb41f5a4ce49,None
2018-07-31 00:00:00+08:00,Equity(1 [1216]),1216,100,2018-07-31 13:30:00+08:00,80.8,76c72d12b06e4e469c7a6cc20e5a0f83,None


## 2

但如果on_error設定成'fail'，整個程式會被中止，然後顯示一樣的錯誤訊息。

In [15]:
def initialize(context):
    context.i = 0
    set_do_not_order_list(restricted_list = [symbol('1101')], on_error='fail')
    set_slippage(slippage.FixedSlippage(spread = 0.0))
    set_commission(commission.PerDollar(cost=commission_cost))
    set_benchmark(symbol('IR0001'))
    
def handle_data(context, data):
    
    if context.i == 0:
        order(symbol('2330'), 100)
        
    if context.i == 2:
        order(symbol('1101'), 100)

    if context.i == 4:
        order(symbol('1216'), 100)
        
    context.i += 1
    
def analyze(context, perf):

    pass

commission_cost = 0.001425
capital_base = 1e6
treasury_returns = get_Treasury_Return(start = start_dt,
                                      end = end_dt,
                                      rate_type = 'Time_Deposit_Rate',                     
                                      term = '1y',
                                      symbol = '5844')

performance = run_algorithm(start=start_dt,
                            end=end_dt,
                            initialize=initialize,
                            handle_data=handle_data,
                            capital_base=capital_base,
                            analyze=analyze,
                            treasury_returns=treasury_returns,
                            trading_calendar=get_calendar(calendar_name),
                            bundle=bundle_name)

positions, transactions, orders = get_transaction_detail(performance)

TradingControlViolation: Order for 100 shares of Equity(0 [1101]) at 2018-07-26 05:30:00+00:00 violates trading constraint
RestrictedListOrder({}).

# set_long_only

### zipline.api.set_long_only(self, on_error='fail')
- Set a rule specifying that this algorithm cannot take short positions.

set_long_only限制所有股票都只能在long position，在這個範例，在持有1000股時賣出500是允許的，但是如果再賣出800，會變成-300 short position，所以就跳出了警示：

    ERROR: handle_violation: Order for -800 shares of Equity(14 [2330]) at 2018-07-30 05:30:00+00:00 violates trading 
    constraint LongOnly({})
   
但因為on_error設定是'log'，股票依然成功賣出，進入short position.

In [16]:
def initialize(context):
    context.i = 0
    set_long_only(on_error='log')
    set_slippage(slippage.FixedSlippage(spread = 0.0))
    set_commission(commission.PerDollar(cost=commission_cost))
    set_benchmark(symbol('IR0001'))
    
def handle_data(context, data):
    
    if context.i == 0:
        order(symbol('2330'), 1000)
        
    if context.i == 2:
        order(symbol('2330'), -500)

    if context.i == 4:
        order(symbol('2330'), -800)
          
    context.i += 1
    
def analyze(context, perf):

    pass

commission_cost = 0.001425
capital_base = 1e6
treasury_returns = get_Treasury_Return(start = start_dt,
                                      end = end_dt,
                                      rate_type = 'Time_Deposit_Rate',                     
                                      term = '1y',
                                      symbol = '5844')

performance = run_algorithm(start=start_dt,
                            end=end_dt,
                            initialize=initialize,
                            handle_data=handle_data,
                            capital_base=capital_base,
                            analyze=analyze,
                            treasury_returns=treasury_returns,
                            trading_calendar=get_calendar(calendar_name),
                            bundle=bundle_name)

positions, transactions, orders = get_transaction_detail(performance)

[2023-08-15 01:10:14.197005]: ERROR: handle_violation: Order for -800 shares of Equity(3 [2330]) at 2018-07-30 05:30:00+00:00 violates trading constraint LongOnly({})
[2023-08-15 01:10:14.230721]: INFO: handle_simulation_end: Simulated 24 trading days
first open: 2018-07-24 01:01:00+00:00
last close: 2018-08-24 05:30:00+00:00


In [17]:
positions[0:6]

,,,amount,cost_basis,last_sale_price
date,asset,symbol,,,
2018-07-25 00:00:00+08:00,Equity(3 [2330]),2330,1000,240.842713,240.5
2018-07-26 00:00:00+08:00,Equity(3 [2330]),2330,1000,240.842713,241.0
2018-07-27 00:00:00+08:00,Equity(3 [2330]),2330,500,241.191125,244.5
2018-07-30 00:00:00+08:00,Equity(3 [2330]),2330,500,241.191125,245.5
2018-07-31 00:00:00+08:00,Equity(3 [2330]),2330,-300,245.065200,246.0
2018-08-01 00:00:00+08:00,Equity(3 [2330]),2330,-300,245.065200,248.0


# set_max_leverage

### zipline.api.set_max_leverage(self, max_leverage)
- max_leverage (float) – The maximum leverage for the algorithm. If not provided there will be no maximum.

這個leverage指的是gross leverage，而且因為程式設計關係，只能直接fail，沒辦法選log。

在這個範例，第一天(7/24)先long 一百萬2330股票，short 一百萬2317，第二日(7/25)成交。第二日的leverage計算方法就是：手上持有各種股票(的絕對值) 乘以當天收盤價，除以當天portfolio_value (雖然一個long一個short剛好抵銷，但因為有手續費，所以會稍微低於起始資金一百萬)
    
( 4149 * 240.5 + 11737 * 82.7 ) / 997194.9 = 1.974022 (參考positions還有portfolio_value 7/25)

假設我們設了一個set_max_leverage(2.0)，在7/26時因為股價波動，leverage超過2.0，程式就會被終止，跳出錯誤訊息。

那如果設2.4，在第三天long 50萬的2454，leverage達到2.48，也一樣會被終止，跳出錯誤訊息。

In [18]:
def initialize(context):
    context.i = 0
    set_slippage(slippage.FixedSlippage(spread = 0.0))
    set_max_leverage(3.0)
    set_commission(commission.PerDollar(cost=commission_cost))
    set_benchmark(symbol('IR0001'))
    
def handle_data(context, data):
    
    if context.i == 0:
        order_value(symbol('2330'), 1e6)
        order_value(symbol('2317'), -1e6)
            
    if context.i == 2:
        order_value(symbol('2454'), 5e5)

    context.i += 1
    
def analyze(context, perf):

    pass

commission_cost = 0.001425
capital_base = 1e6
treasury_returns = get_Treasury_Return(start = start_dt,
                                      end = end_dt,
                                      rate_type = 'Time_Deposit_Rate',                     
                                      term = '1y',
                                      symbol = '5844')

performance = run_algorithm(start=start_dt,
                            end=end_dt,
                            initialize=initialize,
                            handle_data=handle_data,
                            capital_base=capital_base,
                            analyze=analyze,
                            treasury_returns=treasury_returns,
                            trading_calendar=get_calendar(calendar_name),
                            bundle=bundle_name)

positions, transactions, orders = get_transaction_detail(performance)

[2023-08-15 01:10:20.754621]: INFO: handle_simulation_end: Simulated 24 trading days
first open: 2018-07-24 01:01:00+00:00
last close: 2018-08-24 05:30:00+00:00


In [19]:
performance['gross_leverage'][0:4]

2018-07-24 13:30:00+08:00    0.000000
2018-07-25 13:30:00+08:00    1.974022
2018-07-26 13:30:00+08:00    2.000191
2018-07-27 13:30:00+08:00    2.481604
Name: gross_leverage, dtype: float64

In [20]:
positions[0:10]

amount  cost_basis  \
date                      asset            symbol                       
2018-07-25 00:00:00+08:00 Equity(3 [2330]) 2330      4149  240.842712   
                          Equity(2 [2317]) 2317    -11737   82.582153   
2018-07-26 00:00:00+08:00 Equity(3 [2330]) 2330      4149  240.842712   
                          Equity(2 [2317]) 2317    -11737   82.582153   
2018-07-27 00:00:00+08:00 Equity(3 [2330]) 2330      4149  240.842712   
                          Equity(2 [2317]) 2317    -11737   82.582153   
                          Equity(5 [2454]) 2454      1901  267.881188   
2018-07-30 00:00:00+08:00 Equity(3 [2330]) 2330      4149  240.842712   
                          Equity(2 [2317]) 2317    -11737   82.582153   
                          Equity(5 [2454]) 2454      1901  267.881188   

                                                   last_sale_price  
date                      asset            symbol                   
2018-07-25 00:00:00+08:00 Equity(3 [2330]) 2330              240.5  
                          Equity(2 [2317]) 2317               82.7  
2018-07-26 00:00:00+08:00 Equity(3 [2330]) 2330              241.0  
                          Equity(2 [2317]) 2317               83.5  
2018-07-27 00:00:00+08:00 Equity(3 [2330]) 2330              244.5  
                          Equity(2 [2317]) 2317               83.2  
                          Equity(5 [2454]) 2454              267.5  
2018-07-30 00:00:00+08:00 Equity(3 [2330]) 2330              245.5  
                          Equity(2 [2317]) 2317               84.2  
                          Equity(5 [2454]) 2454              260.0

In [21]:
performance['portfolio_value'][0:4]

2018-07-24 13:30:00+08:00    1.000000e+06
2018-07-25 13:30:00+08:00    9.971949e+05
2018-07-26 13:30:00+08:00    9.898798e+05
2018-07-27 13:30:00+08:00    1.007198e+06
Name: portfolio_value, dtype: float64

# set_max_order_count

### zipline.api.set_max_order_count(self, max_count, on_error='fail')
- max_count (int) – The maximum number of orders that can be placed on any single day.

set_max_order_count(max_count, on_error='fail') 用max_count限制一天最多能下幾單，on_error用法同上。

值得注意的是，如果被拆成好幾單，只算第一單。例如下面例子，第一天下單大量2330和2357，因為VolumeShareSlippage的限制，所以拆成數單，導致第三天 (7/26) 不只有第二天訂的三支股票，還有2330和2357的單，共五單。但是程式沒有跳出任何錯誤或警告，因為他判定方法是，第一天下兩單，第二天下三單，都沒有超過3，所以沒有問題。

In [22]:
def initialize(context):
    context.i = 0
    set_slippage(slippage.VolumeShareSlippage(volume_limit=0.025, price_impact=0.1))
    set_max_order_count(max_count=3, on_error='log')
    set_commission(commission.PerDollar(cost=commission_cost))
    set_benchmark(symbol('IR0001'))
    
def handle_data(context, data):
    
    if context.i == 0:
        order_value(symbol('2330'), 5e8)
        order_value(symbol('2357'), 3e8)
            
    if context.i == 1:
        order_value(symbol('2454'), 5e5)
        order_value(symbol('2317'), 5e5)
        order_value(symbol('1101'), 5e5)

    context.i += 1
    
def analyze(context, perf):

    pass

commission_cost = 0.001425
capital_base = 1e9
treasury_returns = get_Treasury_Return(start = start_dt,
                                      end = end_dt,
                                      rate_type = 'Time_Deposit_Rate',                     
                                      term = '1y',
                                      symbol = '5844')

performance = run_algorithm(start=start_dt,
                            end=end_dt,
                            initialize=initialize,
                            handle_data=handle_data,
                            capital_base=capital_base,
                            analyze=analyze,
                            treasury_returns=treasury_returns,
                            trading_calendar=get_calendar(calendar_name),
                            bundle=bundle_name)

positions, transactions, orders = get_transaction_detail(performance)

[2023-08-15 01:10:38.080805]: INFO: handle_simulation_end: Simulated 24 trading days
first open: 2018-07-24 01:01:00+00:00
last close: 2018-08-24 05:30:00+00:00


In [23]:
transactions[0:9]

amount  \
date                      asset            symbol           
2018-07-25 00:00:00+08:00 Equity(3 [2330]) 2330    538250   
                          Equity(4 [2357]) 2357     53175   
2018-07-26 00:00:00+08:00 Equity(3 [2330]) 2330    771400   
                          Equity(4 [2357]) 2357     42475   
                          Equity(5 [2454]) 2454      1904   
                          Equity(2 [2317]) 2317      6045   
                          Equity(0 [1101]) 1101     12196   
2018-07-27 00:00:00+08:00 Equity(3 [2330]) 2330    678225   
                          Equity(4 [2357]) 2357     49750   

                                                                         dt  \
date                      asset            symbol                             
2018-07-25 00:00:00+08:00 Equity(3 [2330]) 2330   2018-07-25 13:30:00+08:00   
                          Equity(4 [2357]) 2357   2018-07-25 13:30:00+08:00   
2018-07-26 00:00:00+08:00 Equity(3 [2330]) 2330   2018-07-26 13:30:00+08:00   
                          Equity(4 [2357]) 2357   2018-07-26 13:30:00+08:00   
                          Equity(5 [2454]) 2454   2018-07-26 13:30:00+08:00   
                          Equity(2 [2317]) 2317   2018-07-26 13:30:00+08:00   
                          Equity(0 [1101]) 1101   2018-07-26 13:30:00+08:00   
2018-07-27 00:00:00+08:00 Equity(3 [2330]) 2330   2018-07-27 13:30:00+08:00   
                          Equity(4 [2357]) 2357   2018-07-27 13:30:00+08:00   

                                                        price  \
date                      asset            symbol               
2018-07-25 00:00:00+08:00 Equity(3 [2330]) 2330    240.515031   
                          Equity(4 [2357]) 2357    261.016313   
2018-07-26 00:00:00+08:00 Equity(3 [2330]) 2330    241.015062   
                          Equity(4 [2357]) 2357    261.016313   
                          Equity(5 [2454]) 2454    263.000002   
                          Equity(2 [2317]) 2317     83.500000   
                          Equity(0 [1101]) 1101     40.500000   
2018-07-27 00:00:00+08:00 Equity(3 [2330]) 2330    244.515281   
                          Equity(4 [2357]) 2357    261.016313   

                                                                           order_id  \
date                      asset            symbol                                     
2018-07-25 00:00:00+08:00 Equity(3 [2330]) 2330    f5ea34189e08424389f3e92e46581104   
                          Equity(4 [2357]) 2357    b6ee53f7757a405fbcb63c19a8d8ffb5   
2018-07-26 00:00:00+08:00 Equity(3 [2330]) 2330    f5ea34189e08424389f3e92e46581104   
                          Equity(4 [2357]) 2357    b6ee53f7757a405fbcb63c19a8d8ffb5   
                          Equity(5 [2454]) 2454    e7422f35c96648f9965adeaacf56d2b9   
                          Equity(2 [2317]) 2317    b533bff4fb114c6681d1b2705b02854f   
                          Equity(0 [1101]) 1101    c15b634897b84043802a76e2a532696d   
2018-07-27 00:00:00+08:00 Equity(3 [2330]) 2330    f5ea34189e08424389f3e92e46581104   
                          Equity(4 [2357]) 2357    b6ee53f7757a405fbcb63c19a8d8ffb5   

                                                  commission  
date                      asset            symbol             
2018-07-25 00:00:00+08:00 Equity(3 [2330]) 2330         None  
                          Equity(4 [2357]) 2357         None  
2018-07-26 00:00:00+08:00 Equity(3 [2330]) 2330         None  
                          Equity(4 [2357]) 2357         None  
                          Equity(5 [2454]) 2454         None  
                          Equity(2 [2317]) 2317         None  
                          Equity(0 [1101]) 1101         None  
2018-07-27 00:00:00+08:00 Equity(3 [2330]) 2330         None  
                          Equity(4 [2357]) 2357         None

# set_max_order_size

### zipline.api.set_max_order_size(self, asset=None, max_shares=None, max_notional=None, on_error='fail')
- asset (Asset, optional) – If provided, this sets the guard only on positions in the given asset.
- max_shares (int, optional) – The maximum number of shares that can be ordered at one time.
- max_notional (float, optional) – The maximum value that can be ordered at one time.

這個函數限制特定股票的單次交易股數和金額，使用方法是：
set_max_order_size(asset=None, max_shares=None, max_notional=None, on_error='fail')

asset不填則是限制所有股票，max_shares和max_notional可以只填一個，如果兩個都填，只要違反一個就會跳出警告或終止程式。

注意：max_shares是依下單時為準，notional計算方法是下單時的張數 * 當天收盤價，所以成交時的股數和金額可能還是會超過限制，細節在下面的範例說明。

In [27]:
def initialize(context):
    context.i = 0
    set_max_order_size(asset= symbol('1101'), max_shares=1000, on_error='log')
    set_max_order_size(asset= symbol('2330'), max_shares=2000, max_notional=481000, on_error='log')
    set_slippage(slippage.FixedSlippage(spread = 0.0))
    set_commission(commission.PerDollar(cost=0.01))
    set_benchmark(symbol('IR0001'))
    
def handle_data(context, data):

    if context.i == 1:
        order(symbol('1101'), 1000)
        order(symbol('2330'), 2000)

    if context.i == 17:
        order(symbol('2330'), 2005)

    context.i += 1
    
def analyze(context, perf):

    pass

commission_cost = 0.001425
capital_base = 1e6
treasury_returns = get_Treasury_Return(start = start_dt,
                                      end = end_dt,
                                      rate_type = 'Time_Deposit_Rate',                     
                                      term = '1y',
                                      symbol = '5844')

performance = run_algorithm(start=start_dt,
                            end=end_dt,
                            initialize=initialize,
                            handle_data=handle_data,
                            capital_base=capital_base,
                            analyze=analyze,
                            treasury_returns=treasury_returns,
                            trading_calendar=get_calendar(calendar_name),
                            bundle=bundle_name)

closing_price = tejapi.get('TWN/APIPRCD',coid=['1101','2330'], opts={'columns':['mdate','coid','close_d']}, mdate={'gte':start_dt,'lte':end_dt }, paginate=True)

positions, transactions, orders = get_transaction_detail(performance)

[2023-08-15 01:11:39.652536]: ERROR: handle_violation: Order for 2005 shares of Equity(3 [2330]) at 2018-08-16 05:30:00+00:00 violates trading constraint MaxOrderSize({'asset': Equity(3 [2330]), 'max_shares': 2000, 'max_notional': 481000})
[2023-08-15 01:11:39.710837]: INFO: handle_simulation_end: Simulated 24 trading days
first open: 2018-07-24 01:01:00+00:00
last close: 2018-08-24 05:30:00+00:00


## 1

限制1101一次只能買1000股，7/25下單1000股，但7/26遇到split，調整後實際上買了1100股，但是因為下單時是1000，所以沒有問題。

In [28]:
performance['orders'][1][0]

{'id': '5a21ef8df55744b386f3c91111b78d45',
 'dt': Timestamp('2018-07-25 13:30:00+0800', tz='Asia/Taipei'),
 'reason': None,
 'created': Timestamp('2018-07-25 13:30:00+0800', tz='Asia/Taipei'),
 'amount': 1000,
 'filled': 0,
 'commission': 0,
 'stop': None,
 'limit': None,
 'stop_reached': False,
 'limit_reached': False,
 'sid': Equity(0 [1101]),
 'status': <ORDER_STATUS.OPEN: 0>}

In [29]:
performance['orders'][2][0]

{'id': '5a21ef8df55744b386f3c91111b78d45',
 'dt': Timestamp('2018-07-26 13:30:00+0800', tz='Asia/Taipei'),
 'reason': None,
 'created': Timestamp('2018-07-25 13:30:00+0800', tz='Asia/Taipei'),
 'amount': 1100,
 'filled': 1100,
 'commission': 445.50000000000006,
 'stop': None,
 'limit': None,
 'stop_reached': False,
 'limit_reached': False,
 'sid': Equity(0 [1101]),
 'status': <ORDER_STATUS.FILLED: 1>}

## 2

在7/25下訂2000股2330，notional = 2000 * 當天收盤240.5 = 481000，符合設定的兩個限制，但7/26成交時，是以241成交，notional = 2000 * 241 = 482000。

In [30]:
closing_price.loc[closing_price['coid'] == '2330'][0:4]

,mdate,coid,close_d
None,,,
24,2018-07-24,2330,241.0
25,2018-07-25,2330,240.5
26,2018-07-26,2330,241.0
27,2018-07-27,2330,244.5


In [31]:
transactions

amount  \
date                      asset            symbol           
2018-07-26 00:00:00+08:00 Equity(0 [1101]) 1101      1100   
                          Equity(3 [2330]) 2330      2000   
2018-08-17 00:00:00+08:00 Equity(3 [2330]) 2330      2005   

                                                                         dt  \
date                      asset            symbol                             
2018-07-26 00:00:00+08:00 Equity(0 [1101]) 1101   2018-07-26 13:30:00+08:00   
                          Equity(3 [2330]) 2330   2018-07-26 13:30:00+08:00   
2018-08-17 00:00:00+08:00 Equity(3 [2330]) 2330   2018-08-17 13:30:00+08:00   

                                                   price  \
date                      asset            symbol          
2018-07-26 00:00:00+08:00 Equity(0 [1101]) 1101     40.5   
                          Equity(3 [2330]) 2330    241.0   
2018-08-17 00:00:00+08:00 Equity(3 [2330]) 2330    239.5   

                                                                           order_id  \
date                      asset            symbol                                     
2018-07-26 00:00:00+08:00 Equity(0 [1101]) 1101    5a21ef8df55744b386f3c91111b78d45   
                          Equity(3 [2330]) 2330    ce2f2ce76f9249de85071b361a6c677e   
2018-08-17 00:00:00+08:00 Equity(3 [2330]) 2330    7a346a6fb5c24b118d0abffc1e236c24   

                                                  commission  
date                      asset            symbol             
2018-07-26 00:00:00+08:00 Equity(0 [1101]) 1101         None  
                          Equity(3 [2330]) 2330         None  
2018-08-17 00:00:00+08:00 Equity(3 [2330]) 2330         None

## 3

在8/16訂2005股，雖然當天notional 239 * 2005 = 479195 沒有超過481000，已經超過2000股的限制。因為on_error = 'log'，程式繼續運作，照樣成交，但是跳出錯誤訊息：
    
    ERROR: handle_violation: Order for 2005 shares of Equity(14 [2330]) at 2018-08-16 05:30:00+00:00 violates trading  
    constraint MaxOrderSize({'asset': Equity(14 [2330]), 'max_shares': 2000, 'max_notional': 481000})
    
    

In [32]:
closing_price.loc[closing_price['coid']=='2330'][17:19]

,mdate,coid,close_d
None,,,
41,2018-08-16,2330,239.0
42,2018-08-17,2330,239.5


# set_max_position_size

### zipline.api.set_max_position_size(self, asset=None, max_shares=None, max_notional=None, on_error='fail')
- asset (Asset, optional) – If provided, this sets the guard only on positions in the given asset.

- max_shares (int, optional) – The maximum number of shares to hold for an asset.

- max_notional (float, optional) – The maximum value to hold for an asset.

這個函數用法跟set_max_order_size非常類似，差別是它是限制某支(或全部)股票的整體股數、市值。

注意：這函數只會檢查下單當下，這特定的單子會不會讓手上的position超過限制，並不是一直追蹤手上position。下面用一些比較特殊的例子，同時應用max_order_size和max_position_size來解釋運作規則。

In [34]:
def initialize(context):
    context.i = 0
    set_max_order_size(asset= symbol('2330'), max_shares=2000, max_notional=481000, on_error='log')
    set_max_position_size(asset= symbol('1101'), max_shares=1050, on_error='log')
    set_max_position_size(asset= symbol('2330'), max_shares=2000, max_notional=600000, on_error='log')
    set_slippage(slippage.FixedSlippage(spread = 0.0))
    set_commission(commission.PerDollar(cost=0.01))
    set_benchmark(symbol('IR0001'))
    
def handle_data(context, data):

    if context.i ==0:
        order(symbol('1101'), 1000)
    
    if context.i == 1:
        order(symbol('2330'), 2000)
        order(symbol('2330'), 1000)

    if context.i == 5:
        order(symbol('2330'), 500)
        
    context.i += 1
    
def analyze(context, perf):

    pass

commission_cost = 0.001425
capital_base = 1e6
treasury_returns = get_Treasury_Return(start = start_dt,
                                      end = end_dt,
                                      rate_type = 'Time_Deposit_Rate',                     
                                      term = '1y',
                                      symbol = '5844')

performance = run_algorithm(start=start_dt,
                            end=end_dt,
                            initialize=initialize,
                            handle_data=handle_data,
                            capital_base=capital_base,
                            analyze=analyze,
                            treasury_returns=treasury_returns,
                            trading_calendar=get_calendar(calendar_name),
                            bundle=bundle_name)

closing_price = tejapi.get('TWN/APIPRCD',coid=['1101','2330'], opts={'columns':['mdate','coid','close_d']}, mdate={'gte':start_dt,'lte':end_dt }, paginate=True)

positions, transactions, orders = get_transaction_detail(performance)

[2023-08-15 01:12:26.561058]: INFO: earn_dividends: Equity(0 [1101]), cash_dividend amount: 1.5, pay_date: 2018-08-24, div_owed: 1500.0
[2023-08-15 01:12:26.561058]: INFO: handle_split: after split: asset: Equity(0 [1101]), amount: 1100, cost_basis: 41.4, last_sale_price: 45.1
[2023-08-15 01:12:26.561058]: INFO: handle_split: returning cash: 7.33
[2023-08-15 01:12:26.580792]: ERROR: handle_violation: Order for 500 shares of Equity(3 [2330]) at 2018-07-31 05:30:00+00:00 violates trading constraint MaxPositionSize({'asset': Equity(3 [2330]), 'max_shares': 2000, 'max_notional': 600000})
[2023-08-15 01:12:26.592605]: ERROR: handle_violation: Order for 500 shares of Equity(3 [2330]) at 2018-07-31 05:30:00+00:00 violates trading constraint MaxPositionSize({'asset': Equity(3 [2330]), 'max_shares': 2000, 'max_notional': 600000})
[2023-08-15 01:12:26.645101]: INFO: handle_simulation_end: Simulated 24 trading days
first open: 2018-07-24 01:01:00+00:00
last close: 2018-08-24 05:30:00+00:00


## 1

雖然原本持有的1000股1101在split之後變成1100股，超過限制的1050股，但沒有錯誤訊息。因為在下單時，系統判定，買1000股，讓手上position從0變成1000股，沒有超過1050。同樣道理，如果單子因為split造成成交股數和下單時股數不同，還是以下單時股數為主。

In [35]:
positions[0:2]

,,,amount,cost_basis,last_sale_price
date,asset,symbol,,,
2018-07-25 00:00:00+08:00,Equity(0 [1101]),1101,1000,45.551,45.1
2018-07-26 00:00:00+08:00,Equity(0 [1101]),1101,1100,41.400,40.5


## 2

在7/25連續下了兩單，分別是2000股和1000股2330，雖然加起來已經3000股，超過限制的2000，但也沒有錯誤訊息。因為兩單都沒有超過max_order_size的2000股限制，而因為兩張隔天才會成交，下單時position = 0，max_position_size認為一單是讓position從0變成2000，另一單0變成1000，都沒有超過總股數2000的限制 (max_notional部分也是同樣概念)。接下來幾天股價上升，乘以手上的3000股，notional也早就超過600000，但也沒有錯誤訊息。

直到7/31下單500股2330時才跳出錯誤訊息，因為手上3000股，再加500，就會超過2000 (max_notional部分也是同樣概念)。因為超過了兩個限制，同一筆交易跳出兩行錯誤訊息 (但因為我們用on_error = 'log'，所有訂單還是成交)。

In [36]:
closing_price.loc[closing_price['coid'] == '2330'][0:9]

,mdate,coid,close_d
None,,,
24,2018-07-24,2330,241.0
25,2018-07-25,2330,240.5
26,2018-07-26,2330,241.0
27,2018-07-27,2330,244.5
28,2018-07-30,2330,245.5
29,2018-07-31,2330,246.0
30,2018-08-01,2330,248.0
31,2018-08-02,2330,244.5
32,2018-08-03,2330,247.0
